In [17]:
import pandas as pd
from bs4 import BeautifulSoup

In [18]:
df = pd.read_csv("hackernews.csv", parse_dates=["time_ts"])
df

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,49756,joe,joe,1188849367,2007-09-03 19:56:07+00:00,"Sure, block Firefox. Guess what I predict will eventually happen? Adblock Plus will offer to install the User Agent Switcher plugin (<a href=""https://addons.mozilla.org/firefox/addon/59"" rel=""nofollow"">https://addons.mozilla.org/firefox/addon/59</a>) after it configures itself.<p>Blocking ads is completely ethical (IMHO). The advertising-based business model is not a contract to which the end user has agreed. People have always had the freedom to turn off the TV or radio during commercial breaks. Automating the process (a la TiVo) at the user's request does not somehow infringe upon some given right on the part of the advertiser to show the user its ads.<p>Just my two cents.<p>Edit: If you're going to downvote me, at least explain why. Kthx.",49691,NaN,NaN,0
1,6168912,jf22,jf22,1375820650,2013-08-06 20:24:10+00:00,I predict somebody will create a killer javascript ide which which will propel the language forward like a rocket booster.<p>Good js is really really hard and it will take a killer tool to make all the poor and mediocre devs start producing awesome software.,6168467,NaN,NaN,0
2,9780434,pzxc,pzxc,1435261286,2015-06-25 19:41:26+00:00,"I&#x27;m going through this right now as well. I ran my own business for 7 years, making content websites monetized through google adsense, building games, and freelancing, among other things. The primary source of revenue was adsense, and it paid the bills enough for me not to need a &quot;real job&quot; for years. 18 months ago, the adsense revenue started to dry up, and in an effort to make some additional income, I started looking for a normal job again.<p>I ended up reluctantly taking a job working for the State of California, mostly because the schedule was flexible and it was a 5 minute commute. In the last 18 months I&#x27;ve gotten two promotions, including one four months after I started which is unheard of in state service (and which my bosses had to fight HR to get). I spearheaded the acquisition and implemented of a version control system, which no one hear had ever used (they were just FTPing files all the time, risking overwriting of others&#x27; work etc), and got b...",9780243,NaN,NaN,0
3,5909557,comex,comex,1371690357,2013-06-20 01:05:57+00:00,"You mean like the microphone found in almost every laptop, tablet, or phone?<p>I guess the difference with this microphone is that it&#x27;s listening to you all the time by default, and that creeps some people out. But it&#x27;s not sending your words over the network, and is not more likely to start doing so than any of the aforementioned devices (actually, much less, because of increased security); it&#x27;s just pattern matching for the word &quot;Xbox&quot;. Kind of like the initial worries over Gmail &quot;reading your email&quot; to target ads: I predict that this too will peter out once people start actually using the product, albeit perhaps never die completely.<p>(For the record, the fact that Gmail messages can also be read by law enforcement under some circumstances is both a serious issue and irrelevant to the analogy.)",5909529,NaN,NaN,0
4,10303830,jules,jules,1443620387,2015-09-30 13:39:47+00:00,"Why would dependent types not extend beyond pure functions? Idris has both dependent types and non pure functions, is that not satisfactory? The fact that you <i>can</i> write software that provably satisfies a strict specification in dependently typed languages doesn&#x27;t mean that you <i>must</i>. It is perfectly possible to program in them like you would in a normal language. That&#x27;s what I was trying to get at: dependent types are also more expressive, not just more constraining. Now that I know about dependent types I regularly run into situations where I think &quot;gosh, if I had dependent types I wouldn&#x27;t have had to put in these type casts or handle these error cases that I know for sure 

In [19]:
def extract_prediction(text):
    soup = BeautifulSoup(text)
    text = soup.text
    text = " ".join(text.split())
    pred_idx = text.find("I predict")
    dot_idx = text[pred_idx:].find(".")
    
    return text[pred_idx:pred_idx + dot_idx + 1]

In [20]:
df["prediction"] = df["text"].apply(extract_prediction)
df["year"] = df["time_ts"].dt.year
df = df[df["prediction"] != ""]

In [21]:
rnd = df.sample(1)
print(f"\"{rnd['prediction'].values[0]}\" -{rnd['by'].values[0]} ({rnd['year'].values[0]})")

"I predict that new APIs won't even be callable anymore from Objective-C in the future." -chriseidhof (2014)


In [22]:
import json

generate = False

if generate:
    for i, d in enumerate(df[["prediction", "author", "year"]].to_dict(orient="records")):
        with open(f"../predictions/{i}", "w") as f:
            json.dump(d, f, ensure_ascii=False, indent=4)
